<div align="center">

# Final Report title here

<div align="center">

### by Jeanette Schulz 2021-11-29

<hr style="border:2px solid black"> </hr>

# Project Goal
The goal of this project is to identifty the attributes of passengers who survived the Titanic shipwreck in order to make receommendations of the best way to travel on a ship to maximize chance of survival should the ship run into a devastating end.

<hr style="border:2px solid black"> </hr>

# Project Description
The weather is getting more and more extreme and unpredictable. We often think of the Titanic is something in the past that won't happen again. However, these events could become more frequent with the change in the oceanic warming and atmospheric changes. We will analyze the attributes of passengers who were more or less likely to survive, develop a model for predicting survival based on those attributes, and leave with both recommendations for future passengers and predictions of survival for a list of passengers on the next ship leaving (delivered via csv).

<hr style="border:2px solid black"> </hr>

# Initial Questions
Was the phrase "women and children first" just a myth or did they really try to get out women and children first? If so, they would show as more likely to survive.

Do those who travel first class get quicker access to life boats?

Do families with small children get priority access? Even if they at the bottom of the boat in 3rd class?

<hr style="border:2px solid black"> </hr>

In [ ]:
# data dictionary here

Feature Datatype Description
payment_type_id
internet_service_type_id
contract_type_id
customer_id

In [4]:
wrangle.get_telco_data()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,3,1,9962-BFPDU,Female,0,Yes,Yes,1,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.05,20.05,No,Month-to-month,None,Mailed check
7039,2,3,1,9967-ATRFS,Female,0,No,No,19,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.90,367.55,No,Month-to-month,None,Mailed check
7040,4,3,1,9970-QBCDA,Female,0,No,No,6,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.70,129.55,No,Month-to-month,None,Credit card (automatic)
7041,2,3,1,9975-SKRNR,Male,0,No,No,1,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,18.90,18.9,No,Month-to-month,None,Mailed check


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

# Turn Off Pink Boxes 
import warnings
warnings.filterwarnings("ignore")

# Import my own "wrangle" Module 
import wrangle

# Remove Limits On Viewing Dataframes
pd.set_option('display.max_columns', None)

<span style="color:red">

# Wrangle - Using Your modules
- (3) You called the functions to wrangle data in your report and included a markdown cell explaining the steps you took to prepare the data and why.

# Wrangle - Missing Values
Decisions made and reasons are communicated and documented for handling missing values. If you imputed based on computing a value (such as mean, median, etc), that was done after splitting the data, and the value was derived from the training dataset only and then imputed into all 3 datasets. If you filled missing values with 0 or a constant not derived from existing values, that can be done prior to splitting the data.
- (2) You handled missing values by imputing or removing them in a logical and communicated way. You correctly handled missing values using a sound logic that was 
    
</span>

In [2]:
train, validate, test = wrangle.prep_telco(wrangle.get_telco_data())
train.head()

,senior_citizen,tenure,monthly_charges,total_charges,is_male,has_partner,has_dependent,has_phone_service,has_paperless_billing,has_churned,multiple_lines_no_phone_service,has_multiple_lines,online_security_no_internet_service,has_online_security,online_backup_no_internet_service,has_online_backup,device_protection_no_internet_service,has_device_protection,tech_support_no_internet_service,has_tech_support,streaming_tv_no_internet_service,has_streaming_tv,internet_service_type_DSL,internet_service_type_fiber_optic,internet_service_type_None,payment_type_bank_transfer_A,payment_type_credit_card_A,payment_type_electronic_check_M,payment_type_mailed_check_M,month_to_month_contract,one_year_contract,two_year_contract
3219,1,66,108.10,7181.95,0,1,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,0
1649,0,8,68.55,564.35,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0
582,0,57,84.50,4845.40,0,1,1,1,1,0,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1
417,0,65,45.25,2933.95,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1
3351,0,27,101.90,2681.15,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0


# Wrangling the Data
To acquire the titanic data, I used the titanic_db in our mySQL server, and selected all columns from the passengers table. select * from titanic_db.passengers.
 
To clean the data, I did the following:

1. Dummy Vars: Get dummy variables for sex, embarked
2. Drop columns: embarked and sex columns (now dummy vars), deck (almost 700 missing values), class (class and pclass represented the same information, so I kept the one represented as an integer).
3. Drop rows: where age is missing (177 rows are missing values, and age will likely be a very influential feature).

<span style="color:red">

# Explore - Ask a clear question
At least 4 of the questions asked and answered of the data are shared in the final report notebook. You should call out questions of the data using natural language that speaks to the business stakeholders in markdown cells, ideally a header prior to the visualization or statistical test, that you then explore. This does not take the place of stating your null hypothesis/alternative hypothesis when doing a statistical test. But those hypotheses are generally for you. By writing questions that you intend to answer with visualizations and statistical tests in natural language, like "Are office supplies leading to differences in profit in Texas?", you are able to guide both yourself and your reader through the highlights of your analysis. You ask a question, create a visual, run a statistical test (if appropriate), and wrap it nicely with a markdown cell that contains a clear answer in layman's terms. You do all that before moving to the next question.
- (6) Viz's/tests wrapped in a formed Q & A: You called out 4 of the questions you asked of the data and provided a clear answer using natural language in markdown cells in your final report.
</span>